In [1]:
import json
import pandas as pd
from pandas.tseries.resample import TimeGrouper
from pandas.tseries.offsets import DateOffset

In [2]:
tweets_data_path = 'enikos_18_4.txt'

In [3]:
tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue


In [4]:
print (len(tweets_data))

8494


In [5]:
tweets = pd.DataFrame()

In [6]:
tweets['text'] = list(map(lambda tweet: tweet['text'], tweets_data))  
#list (map..) in order to parse inside the array
tweets['created_at'] = list(map(lambda tweet: tweet['created_at'], tweets_data))

In [7]:
tweets.head()

,text,created_at
0,"Ξεκινάμε #stoneniko με τον @NChatzinikolaou, σ...",Mon Apr 18 20:25:02 +0000 2016
1,RT @enikos_gr: Ξεκινάμε #stoneniko με τον @NCh...,Mon Apr 18 20:25:15 +0000 2016
2,Eεχει αποψε #enikos,Mon Apr 18 20:25:20 +0000 2016
3,Μπορείτε να παρακολουθήσετε την εκπομπή του @N...,Mon Apr 18 20:25:25 +0000 2016
4,RT @enikos_gr: Μπορείτε να παρακολουθήσετε την...,Mon Apr 18 20:25:30 +0000 2016


In [8]:
tweets['created_at'] = pd.to_datetime(pd.Series(tweets['created_at']))
tweets.set_index('created_at', drop=False, inplace=True)
tweets.index = tweets.index.tz_localize('GMT').tz_convert('Europe/Athens')
tweets.index = tweets.index - DateOffset(hours = 24) #isws auto prepei na ginei 24 hours
tweets.index

DatetimeIndex(['2016-04-17 23:25:02+03:00', '2016-04-17 23:25:15+03:00',
               '2016-04-17 23:25:20+03:00', '2016-04-17 23:25:25+03:00',
               '2016-04-17 23:25:30+03:00', '2016-04-17 23:25:35+03:00',
               '2016-04-17 23:25:39+03:00', '2016-04-17 23:25:56+03:00',
               '2016-04-17 23:26:12+03:00', '2016-04-17 23:26:40+03:00',
               ...
               '2016-04-18 02:26:11+03:00', '2016-04-18 02:26:12+03:00',
               '2016-04-18 02:26:13+03:00', '2016-04-18 02:26:17+03:00',
               '2016-04-18 02:26:20+03:00', '2016-04-18 02:26:21+03:00',
               '2016-04-18 02:26:28+03:00', '2016-04-18 02:26:29+03:00',
               '2016-04-18 02:26:34+03:00', '2016-04-18 02:26:34+03:00'],
              dtype='datetime64[ns, Europe/Athens]', name='created_at', length=8494, freq=None)

In [9]:
tweets1m = tweets['created_at'].resample('1t').count()

In [10]:
avg = tweets1m.mean()

In [11]:
avg

46.67032967032967

In [12]:
import vincent
vincent.core.initialize_notebook()
area = vincent.Area(tweets1m)
area.colors(brew='Spectral')
area.display()

In [13]:
print (len(tweets_data))

8494


In [14]:
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist

In [15]:
stop = stopwords.words('greek')
text = tweets['text']

In [16]:
tokens = []
for txt in text.values:
    tokens.extend([t.lower().strip(":,.") for t in txt.split()])

filteredtokens = [w for w in tokens if not w in stop]

In [17]:
#compute frequency distribution
freqdist = nltk.FreqDist(filteredtokens)
#find 100 most frequent words
freqdist = freqdist.most_common(100)
freqdist





[('#enikos', 6632),
 ('είναι', 810),
 ('από', 686),
 ('τους', 654),
 ('μας', 588),
 ('ρε', 583),
 ('@adonisgeorgiadi', 560),
 ('τι', 518),
 ('@nchatzinikolaou', 426),
 ('στα', 415),
 ('τις', 393),
 ('ότι', 390),
 ('σπίρτζης', 377),
 ('έχει', 359),
 ('σπίρτζη', 337),
 ('λέει', 313),
 ('τη', 304),
 ('', 280),
 ('γιατί', 258),
 ('μανταλένα', 251),
 ('σπιρτζης', 244),
 ('αυτό', 237),
 ('#enikos_gr', 232),
 ('όταν', 220),
 ('σας', 215),
 ('ή', 209),
 ('κάνει', 208),
 ('ανελ', 205),
 ('&', 204),
 ('@gveltsi', 200),
 ('συριζα', 194),
 ('τώρα', 187),
 ('ευρώ', 181),
 ('αλλά', 178),
 ('@athandim', 175),
 ('λοβέρδος', 169),
 ('μέτρα', 168),
 ('μόνο', 155),
 ('μιλάει', 153),
 ('-', 152),
 ('αδωνις', 150),
 ('ένα', 149),
 ('άδωνις', 148),
 ('έχουν', 146),
 ('πάνελ', 146),
 ('όχι', 140),
 ('2700', 140),
 ('πασοκ', 137),
 ('πάει', 136),
 ('χώρα', 136),
 ('@htheoharis', 135),
 ('δραχμή', 132),
 ('λεει', 128),
 ('κοινό', 127),
 ('τ', 127),
 ('νδ', 125),
 ('ήταν', 124),
 ('μετά', 124),
 ('α', 123),
 ('

In [18]:
import numpy as np
import lda
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [19]:
tokens = {}

for i in range(len(filteredtokens)):
    tokens[i] = filteredtokens[i]

len(tokens)


88719

In [20]:
len(filteredtokens)

88719

In [21]:
tf = CountVectorizer(strip_accents='unicode', max_df=0.95, min_df=2,stop_words= stopwords.words('greek'))
tfs1 = tf.fit_transform(tokens.values())
num = 3
model = lda.LDA(n_topics=num, n_iter=500, random_state=1)

In [ ]:
#Document Term Matrix structure
model.fit_transform(tfs1)

#Obtain the words with high probabilities
topic_word = model.topic_word_

#Obtain the feature names
vocab = tf.get_feature_names()
tokens.values()

In [28]:
n_top_words =10
for i, tokens in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(tokens)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: τους μανταλενα ρε τη ανελ μιλαει ενα 2700 gveltsi
Topic 1: σπιρτζης adonisgeorgiadi nchatzinikolaou αδωνις τωρα ευρω κανει τι δραχμη
Topic 2: μας εχει σπιρτζη λεει στα τι τις ρε enikos_gr


In [29]:
doc_topic = model.doc_topic_    

for i in range(40):
     print("{} (top topic: {})".format(text[i], doc_topic[i].argmax()))

Ξεκινάμε #stoneniko με τον @NChatzinikolaou, συζητάμε για την πορεία της διαπραγμάτευσης, την αξιολόγηση και το νέο πακέτο μέτρων #enikos (top topic: 0)
RT @enikos_gr: Ξεκινάμε #stoneniko με τον @NChatzinikolaou, συζητάμε για την πορεία της διαπραγμάτευσης, την αξιολόγηση και το νέο πακέτο μ… (top topic: 0)
Eεχει αποψε  #enikos (top topic: 1)
Μπορείτε να παρακολουθήσετε την εκπομπή του @NChatzinikolaou στο https://t.co/UW9dlJLaqD και στο @StarChannelGr #enikos (top topic: 1)
RT @enikos_gr: Μπορείτε να παρακολουθήσετε την εκπομπή του @NChatzinikolaou στο https://t.co/UW9dlJLaqD και στο @StarChannelGr #enikos (top topic: 0)
Ο  Σπίρτζης απο Πρασινοφρουρός Γ Εθνικής spokesman της ΚΥΒΕΡΝΗΣΑΡΑΣ
Για αυτό είστε λίγοι ρε #enikos (top topic: 1)
Καλησπέρα κε Χατζηνικολαου Φαίνεται ότι δεν θα έχουμε συμφωνία ούτε αξιολόγηση με αυτή την κυβέρνηση. Μετά όμως τι; @NChatzinikolaou #enikos (top topic: 1)
αντε να δουμε τι ατακα θα ακουσουμε αποψε #enikos (top topic: 2)
RT @KOPRITHS: Αψουψουψουυυυυυυ
Ας 